In [1]:
#Install Dependencies
import pandas as pd
from pathlib import Path
import re

pd.set_option('display.max_columns',50)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

!pip install openmeteo-requests

!pip install requests-cache retry-requests numpy pandas


import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.utils import resample

In [4]:
#Import CSV's

atlanta_ride_df = pd.read_csv('https://raw.githubusercontent.com/jchay3/james_hay_project_2_Compare_ride_patterns_and_predictive_modeling_of_rides_by-_transport_x/main/Atlanta%20Market%20Rides%20without%20Customer%20Info(1).csv')

In [5]:
#Display the Data
atlanta_ride_df

,ride_id,vehicle_number,ride_type,proximity,driver_name,pickup,radius_schedule_pickup_dispatch_text,pickup_driver_zone,departure,dropoff_driver_zone,ride_start_date_time,ride_end_date_time,customer_mobile,ride_status,allow_start_ride_at,allow_start_ride_description,long_distance_pickup_fee,long_distance_pickup_min_passengers,checkedin_passengers_after_finish_boarding_confirmed_count,age_surge_charge,is_rescan_data_available,total_revenue,ride_miles
0,555847,Fode Soumah,Ride Now,A,Fode Soumah,Midtown,NaN,NaN,729 Brittain Drive Northwest,NaN,03/04/2025 10:23 PM (EST),03/04/2025 10:30 PM (EST),4042858581,Completed,NaN,NaN,0,0,0,0,0,20.64,0.81
1,555751,Fode Soumah,Ride Now,A,Fode Soumah,Midtown,NaN,NaN,North Avenue Apartments,NaN,03/04/2025 08:01 PM (EST),03/04/2025 08:05 PM (EST),6789952504,Completed,NaN,NaN,0,0,0,0,0,16.05,1.19
2,555723,Fode Soumah,Scheduled,A,Fode Soumah,North Avenue Apartments,Radius 1,NaN,Buckhead Fight Club,NaN,03/04/2025 06:42 PM (EST),03/04/2025 06:57 PM (EST),3013370896,Completed,NaN,NaN,0,0,0,0,0,46.80,8.10
3,555626,Fode Soumah,Ride Now,B,Fode Soumah,Home Park,NaN,NaN,955 Spring Street Northwest,NaN,03/03/2025 09:14 PM (EST),03/03/2025 09:21 PM (EST),6178240853,Completed,NaN,NaN,0,0,0,0,0,16.23,1.31
4,555599,Fode Soumah,Ride Now,C,Fode Soumah,Buckhead Fight Club,NaN,NaN,120 North Avenue Northwest,NaN,03/03/2025 08:39 PM (EST),03/03/2025 08:53 PM (EST),4433559115,Completed,NaN,NaN,0,0,0,0,0,52.64,7.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,520226,Fode Soumah,Scheduled,A,Fode Soumah,Clifton Community,Radius 2,NaN,Hudson Grille - Little 5 Points,NaN,02/05/2025 05:53 PM (EST),02/05/2025 06:11 PM (EST),2034612969,Completed,NaN,NaN,0,0,0,0,0,86.66,3.07
185,520093,Fode Soumah,Ride Now,A,Fode Soumah,Clifton Community,NaN,NaN,Smith's Olde Bar,NaN,02/04/2025 11:01 PM (EST),02/04/2025 11:14 PM (EST),6467404455,Completed,NaN,NaN,0,0,0,0,0,48.88,2.87
186,520028,Fode Soumah,Ride Now,B,Fode Soumah,Old Fourth Ward,NaN,NaN,18 Eagle Row,NaN,02/04/2025 10:12 PM (EST),02/04/2025 10:28 PM (EST),6172840818,Completed,NaN,NaN,0,0,0,0,0,39.06,3.98
187,519978,Fode Soumah,Ride Now,B,Fode Soumah,Buckhead,NaN,NaN,Alpha Xi Delta Sorority - Gamma Eta Chapter,NaN,02/04/2025 08:53 PM (EST),02/04/2025 09:06 PM (EST),6789107907,Completed,NaN,NaN,0,0,0,0,0,58.50,2.22


In [6]:
#set up API Client
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [7]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
locations = [
	(30.2672, -97.7431), # Austin, Tx
    (30.4433, -91.1875), # Baton Rouge, LA
    (33.749, -84.388) # Atlanta, GA
]

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 33.749,
	"longitude": -84.388,
	"start_date": "2023-12-31",
	"end_date": "2025-03-01",
	"hourly": ["temperature_2m", "precipitation", "rain", "snowfall", "cloud_cover", "wind_speed_10m"],
	"timezone": "America/Chicago"
}
responses = openmeteo.weather_api(url, params=params)

In [8]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 33.77855682373047°N -84.40298461914062°E
Elevation 327.0 m asl
Timezone b'America/Chicago' b'GMT-5'
Timezone difference to GMT+0 -18000 s


In [9]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()


hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m


atlanta_hourly_dataframe = pd.DataFrame(data = hourly_data)
print(atlanta_hourly_dataframe)

                           date  temperature_2m  precipitation  rain  \
0     2023-12-31 05:00:00+00:00         -0.5035            0.0   0.0   
1     2023-12-31 06:00:00+00:00         -0.7535            0.0   0.0   
2     2023-12-31 07:00:00+00:00         -1.1535            0.0   0.0   
3     2023-12-31 08:00:00+00:00         -1.5535            0.0   0.0   
4     2023-12-31 09:00:00+00:00         -1.6035            0.0   0.0   
...                         ...             ...            ...   ...   
10243 2025-03-02 00:00:00+00:00         12.0965            0.0   0.0   
10244 2025-03-02 01:00:00+00:00         12.1965            0.0   0.0   
10245 2025-03-02 02:00:00+00:00         10.3965            0.0   0.0   
10246 2025-03-02 03:00:00+00:00          7.7465            0.0   0.0   
10247 2025-03-02 04:00:00+00:00          6.0465            0.0   0.0   

       snowfall  cloud_cover  wind_speed_10m  
0           0.0          0.0        7.771331  
1           0.0          0.0        8.049